In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ Libraries imported successfully")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🤖 NumPy version: {np.__version__}")

## 1. Load Data

In [ ]:
# Load crime data
print("Loading LA Crime Data...")
df_raw = pd.read_csv('Crime_Data_from_2020_to_Present.csv')

print(f"✅ Data loaded: {len(df_raw):,} records")
print(f"\n📊 Dataset shape: {df_raw.shape}")
print(f"\n📋 Columns: {list(df_raw.columns)}")
print(f"\n🔍 Sample data:")
df_raw.head()

## 2. Data Cleaning & Preparation

In [ ]:
# Create working copy
df = df_raw.copy()

print("="*80)
print("🧹 DATA CLEANING FOR FORECASTING")
print("="*80)

# Keep only essential columns for forecasting
essential_cols = [
    'DATE OCC',      # Occurrence date
    'TIME OCC',      # Occurrence time
    'AREA',          # Area code
    'AREA NAME',     # Area name
    'Rpt Dist No',   # Reporting district
    'Crm Cd',        # Crime code
    'Crm Cd Desc',   # Crime description
]

df = df[essential_cols].copy()

# Remove duplicates
initial_count = len(df)
df = df.drop_duplicates()
duplicates_removed = initial_count - len(df)

# Remove records with missing core features
df = df.dropna(subset=['DATE OCC', 'AREA', 'Crm Cd Desc'])

print(f"✅ Duplicates removed: {duplicates_removed:,}")
print(f"✅ Records with complete core data: {len(df):,}")
print(f"\n📊 Data retention: {len(df)/len(df_raw)*100:.1f}% of original data")

## 3. Feature Engineering - Temporal Features

In [ ]:
# Parse dates and times
print("="*80)
print("⏰ EXTRACTING TEMPORAL FEATURES")
print("="*80)

df['DATE OCC'] = pd.to_datetime(df['DATE OCC'], errors='coerce')

# Extract date components
df['year'] = df['DATE OCC'].dt.year
df['month'] = df['DATE OCC'].dt.month
df['day'] = df['DATE OCC'].dt.day
df['day_of_week'] = df['DATE OCC'].dt.dayofweek  # 0=Monday, 6=Sunday
df['day_name'] = df['DATE OCC'].dt.day_name()
df['week_of_year'] = df['DATE OCC'].dt.isocalendar().week

# Extract time components (CORRECTED METHOD)
df['TIME_OCC_numeric'] = pd.to_numeric(df['TIME OCC'], errors='coerce')

# Validate time ranges
valid_time_mask = (
    (df['TIME_OCC_numeric'] >= 0) & 
    (df['TIME_OCC_numeric'] <= 2359) &
    ((df['TIME_OCC_numeric'] % 100) < 60)
)

# Extract hour correctly
df['hour'] = np.where(
    valid_time_mask,
    (df['TIME_OCC_numeric'] // 100).astype(int),
    np.nan
)

# Create time blocks (3-hour windows for forecasting)
def get_time_block(hour):
    if pd.isna(hour):
        return 'Unknown'
    elif 0 <= hour < 3:
        return '00-03_Late_Night'
    elif 3 <= hour < 6:
        return '03-06_Early_Morning'
    elif 6 <= hour < 9:
        return '06-09_Morning'
    elif 9 <= hour < 12:
        return '09-12_Late_Morning'
    elif 12 <= hour < 15:
        return '12-15_Afternoon'
    elif 15 <= hour < 18:
        return '15-18_Late_Afternoon'
    elif 18 <= hour < 21:
        return '18-21_Evening'
    else:
        return '21-00_Night'

df['time_block'] = df['hour'].apply(get_time_block)

# Additional temporal features
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_night'] = ((df['hour'] >= 18) | (df['hour'] <= 6)).fillna(0).astype(int)

# Create date key for aggregation
df['date'] = df['DATE OCC'].dt.date

print(f"✅ Temporal features created")
print(f"   Date range: {df['DATE OCC'].min()} to {df['DATE OCC'].max()}")
print(f"   Valid times: {valid_time_mask.sum():,} ({valid_time_mask.sum()/len(df)*100:.1f}%)")
print(f"   Time blocks created: {df['time_block'].nunique()} unique blocks")
print(f"\n📊 Time block distribution:")
print(df['time_block'].value_counts().sort_index())

## 4. Create Crime Categories

In [ ]:
# Identify top crime types
print("="*80)
print("🎯 CREATING CRIME CATEGORIES")
print("="*80)

crime_counts = df['Crm Cd Desc'].value_counts()
top_n = 20
top_crimes = crime_counts.head(top_n).index.tolist()

# Create crime category
df['crime_category'] = df['Crm Cd Desc'].apply(
    lambda x: x if x in top_crimes else 'OTHER'
)

print(f"✅ Created {top_n + 1} crime categories (Top {top_n} + OTHER)")
print(f"\n📊 Top 10 Crime Categories:")
for i, (crime, count) in enumerate(df['crime_category'].value_counts().head(10).items(), 1):
    print(f"   {i:2d}. {crime[:50]:50s} {count:>7,} ({count/len(df)*100:5.1f}%)")

## 5. CRITICAL STEP: Aggregate Data for Forecasting

This is where we transform from individual crime records to forecasting format:
- Group by: Area + Date + Time Block
- Aggregate: Count total crimes, count by type

In [ ]:
print("="*80)
print("🔄 AGGREGATING DATA FOR FORECASTING")
print("="*80)

# Create aggregation keys
df['area_name_clean'] = df['AREA NAME'].fillna('Unknown')

# Aggregate by area, date, and time block
print("Aggregating crime counts by Area + Date + Time Block...")

# Total crime count per area/date/time_block
df_agg = df.groupby(['area_name_clean', 'date', 'time_block']).agg({
    'Crm Cd': 'count',  # Total crimes
    'AREA': 'first',    # Area code
    'year': 'first',
    'month': 'first',
    'day': 'first',
    'day_of_week': 'first',
    'is_weekend': 'first',
}).reset_index()

df_agg = df_agg.rename(columns={'Crm Cd': 'crime_count'})

# Crime type distribution per area/date/time_block
crime_type_pivot = df.groupby(['area_name_clean', 'date', 'time_block', 'crime_category']).size().unstack(fill_value=0)

# Merge back
df_agg = df_agg.merge(
    crime_type_pivot,
    left_on=['area_name_clean', 'date', 'time_block'],
    right_index=True,
    how='left'
)

print(f"\n✅ TRANSFORMATION COMPLETE")
print(f"   Original records: {len(df):,} individual crimes")
print(f"   Aggregated records: {len(df_agg):,} time windows")
print(f"   Unique areas: {df_agg['area_name_clean'].nunique()}")
print(f"   Date range: {df_agg['date'].min()} to {df_agg['date'].max()}")
print(f"   Time blocks: {df_agg['time_block'].nunique()}")

print(f"\n📊 Sample aggregated data:")
df_agg.head(10)

## 6. Feature Engineering for Forecasting Model

In [ ]:
print("="*80)
print("🔧 ENGINEERING FORECASTING FEATURES")
print("="*80)

# Sort by area and date for time-series features
df_agg = df_agg.sort_values(['area_name_clean', 'date', 'time_block'])

# Create lagged features (past crime counts)
print("Creating historical crime features...")

for area in df_agg['area_name_clean'].unique():
    area_mask = df_agg['area_name_clean'] == area
    
    # Lag features (previous day's crime count)
    df_agg.loc[area_mask, 'crime_count_lag1'] = df_agg.loc[area_mask, 'crime_count'].shift(1)
    df_agg.loc[area_mask, 'crime_count_lag7'] = df_agg.loc[area_mask, 'crime_count'].shift(7)
    
    # Rolling averages
    df_agg.loc[area_mask, 'crime_count_rolling_7'] = df_agg.loc[area_mask, 'crime_count'].rolling(window=7, min_periods=1).mean()
    df_agg.loc[area_mask, 'crime_count_rolling_30'] = df_agg.loc[area_mask, 'crime_count'].rolling(window=30, min_periods=1).mean()

# Fill NaN in lag features with 0 (for initial periods)
df_agg[['crime_count_lag1', 'crime_count_lag7', 'crime_count_rolling_7', 'crime_count_rolling_30']] = \
    df_agg[['crime_count_lag1', 'crime_count_lag7', 'crime_count_rolling_7', 'crime_count_rolling_30']].fillna(0)

# Encode categorical variables
le_area = LabelEncoder()
le_timeblock = LabelEncoder()

df_agg['area_encoded'] = le_area.fit_transform(df_agg['area_name_clean'])
df_agg['time_block_encoded'] = le_timeblock.fit_transform(df_agg['time_block'])

print(f"\n✅ Forecasting features created")
print(f"   Lag features: 2 (lag1, lag7)")
print(f"   Rolling features: 2 (7-day, 30-day averages)")
print(f"   Encoded features: 2 (area, time_block)")

print(f"\n📊 Feature summary:")
print(f"   Total features: {df_agg.shape[1]}")
print(f"   Records ready for modeling: {len(df_agg):,}")

## 7. Prepare Training Data

In [ ]:
print("="*80)
print("📦 PREPARING TRAINING DATA")
print("="*80)

# Select features for model
feature_cols = [
    'area_encoded',
    'time_block_encoded',
    'year',
    'month',
    'day',
    'day_of_week',
    'is_weekend',
    'crime_count_lag1',
    'crime_count_lag7',
    'crime_count_rolling_7',
    'crime_count_rolling_30'
]

# Target variable: crime_count
target_col = 'crime_count'

# Remove rows with any NaN values
df_model = df_agg[feature_cols + [target_col]].dropna()

X = df_model[feature_cols]
y = df_model[target_col]

# Time-series split (train on past, test on future)
# Use 80% for training, 20% for testing
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"✅ Data prepared for modeling")
print(f"\n📊 Data Split:")
print(f"   Training set: {len(X_train):,} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Test set: {len(X_test):,} samples ({len(X_test)/len(X)*100:.1f}%)")
print(f"\n📋 Features: {len(feature_cols)}")
for i, col in enumerate(feature_cols, 1):
    print(f"   {i:2d}. {col}")
print(f"\n🎯 Target: {target_col}")
print(f"   Min: {y_train.min()}, Max: {y_train.max()}, Mean: {y_train.mean():.2f}")

## 8. Train XGBoost Regression Model

### Is XGBoost Right for Crime Forecasting?

**YES! XGBoost is EXCELLENT for this use case:**

✅ **Regression Mode** - XGBRegressor predicts crime counts (continuous values)
✅ **Handles Temporal Features** - Excellent with lag features and rolling averages
✅ **Non-Linear Patterns** - Captures complex time/area interactions
✅ **Feature Importance** - Shows which factors drive crime counts
✅ **Fast Prediction** - Real-time forecasting for police deployment

**Alternative Models to Consider:**
- Prophet (Facebook's time-series tool) - Better for pure time-series
- ARIMA/SARIMA - Traditional time-series forecasting
- Random Forest Regressor - Similar performance, more interpretable
- Neural Networks (LSTM) - Better for sequential patterns, needs more data

In [ ]:
print("="*80)
print("🚀 TRAINING XGBOOST REGRESSION MODEL")
print("="*80)

# Initialize XGBoost Regressor
model = XGBRegressor(
    objective='reg:squarederror',  # Regression objective
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

print("Training model...")
model.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)

print("\n✅ Model training complete!")
print(f"\n📊 Model Configuration:")
print(f"   Objective: Crime Count Regression")
print(f"   Trees: {model.n_estimators}")
print(f"   Max depth: {model.max_depth}")
print(f"   Learning rate: {model.learning_rate}")

## 9. Evaluate Model Performance

In [ ]:
print("="*80)
print("📈 MODEL EVALUATION")
print("="*80)

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate metrics
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"\n🎯 REGRESSION METRICS:")
print(f"\n   Training Set:")
print(f"      MAE (Mean Absolute Error): {train_mae:.2f} crimes")
print(f"      RMSE (Root Mean Squared Error): {train_rmse:.2f} crimes")
print(f"      R² Score: {train_r2:.4f}")

print(f"\n   Test Set:")
print(f"      MAE (Mean Absolute Error): {test_mae:.2f} crimes")
print(f"      RMSE (Root Mean Squared Error): {test_rmse:.2f} crimes")
print(f"      R² Score: {test_r2:.4f}")

print(f"\n💡 Interpretation:")
print(f"   On average, predictions are off by ±{test_mae:.1f} crimes")
print(f"   Model explains {test_r2*100:.1f}% of crime count variance")

# Visualize predictions vs actual
plt.figure(figsize=(15, 5))

# Plot 1: Actual vs Predicted
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_test_pred, alpha=0.3, s=10)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Crime Count')
plt.ylabel('Predicted Crime Count')
plt.title('Actual vs Predicted Crime Counts')
plt.grid(True, alpha=0.3)

# Plot 2: Residuals
plt.subplot(1, 2, 2)
residuals = y_test - y_test_pred
plt.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Prediction Error (Actual - Predicted)')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.axvline(x=0, color='r', linestyle='--', lw=2)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ Evaluation complete")

## 10. Feature Importance Analysis

In [ ]:
print("="*80)
print("🔍 FEATURE IMPORTANCE ANALYSIS")
print("="*80)

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n📊 Top Features for Crime Count Prediction:")
for i, row in feature_importance.iterrows():
    print(f"   {row['feature']:30s} {row['importance']:.4f}")

# Visualize
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.xlabel('Importance Score')
plt.title('Feature Importance for Crime Count Prediction')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Feature importance analysis complete")

## 11. Crime Forecasting Interface

This is the key deliverable: A function that takes Area + Time and predicts crime counts

In [ ]:
def predict_crime_count(area_name, date_str, time_block):
    """
    Predict crime count for a given area, date, and time block.
    
    Parameters:
    -----------
    area_name : str
        Area name (e.g., 'Central', 'Hollywood')
    date_str : str
        Date in format 'YYYY-MM-DD'
    time_block : str
        Time block (e.g., '18-21_Evening', '00-03_Late_Night')
    
    Returns:
    --------
    dict : Prediction results with crime count and confidence interval
    """
    
    # Parse input
    date = pd.to_datetime(date_str)
    
    # Check if area exists
    if area_name not in le_area.classes_:
        return {"error": f"Area '{area_name}' not found. Available areas: {list(le_area.classes_)}"}
    
    # Check if time block exists
    if time_block not in le_timeblock.classes_:
        return {"error": f"Time block '{time_block}' not found. Available blocks: {list(le_timeblock.classes_)}"}
    
    # Encode inputs
    area_encoded = le_area.transform([area_name])[0]
    time_block_encoded = le_timeblock.transform([time_block])[0]
    
    # Get historical data for this area and time block
    hist_data = df_agg[
        (df_agg['area_name_clean'] == area_name) &
        (df_agg['time_block'] == time_block)
    ].tail(30)  # Last 30 occurrences
    
    # Calculate lag and rolling features
    crime_count_lag1 = hist_data['crime_count'].iloc[-1] if len(hist_data) > 0 else 0
    crime_count_lag7 = hist_data['crime_count'].iloc[-7] if len(hist_data) >= 7 else 0
    crime_count_rolling_7 = hist_data['crime_count'].tail(7).mean() if len(hist_data) >= 7 else 0
    crime_count_rolling_30 = hist_data['crime_count'].tail(30).mean() if len(hist_data) > 0 else 0
    
    # Create feature vector
    features = pd.DataFrame([{
        'area_encoded': area_encoded,
        'time_block_encoded': time_block_encoded,
        'year': date.year,
        'month': date.month,
        'day': date.day,
        'day_of_week': date.dayofweek,
        'is_weekend': 1 if date.dayofweek >= 5 else 0,
        'crime_count_lag1': crime_count_lag1,
        'crime_count_lag7': crime_count_lag7,
        'crime_count_rolling_7': crime_count_rolling_7,
        'crime_count_rolling_30': crime_count_rolling_30
    }])
    
    # Make prediction
    predicted_count = model.predict(features)[0]
    
    # Calculate confidence interval (±1 MAE)
    lower_bound = max(0, predicted_count - test_mae)
    upper_bound = predicted_count + test_mae
    
    return {
        'area': area_name,
        'date': date_str,
        'time_block': time_block,
        'predicted_crime_count': round(predicted_count, 1),
        'confidence_interval': f"{round(lower_bound, 1)} - {round(upper_bound, 1)} crimes",
        'historical_avg': round(crime_count_rolling_30, 1),
        'trend': 'Increasing' if predicted_count > crime_count_rolling_30 else 'Decreasing'
    }

print("✅ Crime forecasting interface created!")
print("\n📋 Available functions:")
print("   predict_crime_count(area_name, date_str, time_block)")
print("\n💡 Example usage:")
print("   result = predict_crime_count('Central', '2026-01-15', '18-21_Evening')")

## 12. Example Predictions - Test the System

In [ ]:
print("="*80)
print("🎯 EXAMPLE CRIME FORECASTS")
print("="*80)

# Get available areas
available_areas = df_agg['area_name_clean'].unique()[:5]  # First 5 areas

# Test predictions
test_date = '2026-01-15'
test_scenarios = [
    (available_areas[0], test_date, '18-21_Evening'),
    (available_areas[0], test_date, '00-03_Late_Night'),
    (available_areas[1], test_date, '12-15_Afternoon'),
    (available_areas[2], test_date, '06-09_Morning'),
]

print(f"\n📊 Forecasts for {test_date}:\n")

for area, date, time_block in test_scenarios:
    result = predict_crime_count(area, date, time_block)
    
    if 'error' not in result:
        print(f"🔮 {area} - {time_block}")
        print(f"   Expected crimes: {result['predicted_crime_count']}")
        print(f"   Range: {result['confidence_interval']}")
        print(f"   Trend: {result['trend']} (vs avg: {result['historical_avg']})")
        print()

print("="*80)
print("✅ FORECASTING SYSTEM READY FOR DEPLOYMENT")
print("="*80)

## 13. Summary & Next Steps

### What We Built:
✅ **Crime Count Forecasting Model** - Predicts how many crimes will occur
✅ **XGBoost Regression** - Optimized for temporal and spatial patterns
✅ **Prediction Interface** - Easy to use: input area + time → get crime forecast
✅ **Feature Engineering** - Lag features, rolling averages, temporal patterns

### Model Performance:
- Predicts crime counts with reasonable accuracy
- Captures temporal patterns (time of day, day of week)
- Accounts for geographic variation (area differences)
- Uses historical trends for better predictions

### Why XGBoost is Right:
✅ **Regression capability** - Predicts continuous counts
✅ **Handles temporal features** - Excellent with lag and rolling features
✅ **Non-linear patterns** - Captures complex time/area interactions
✅ **Fast inference** - Real-time predictions for police operations
✅ **Feature importance** - Shows what drives crime patterns

### Next Steps:
1. **Crime Type Distribution** - Add model to predict WHAT types of crimes
2. **Hyperparameter Tuning** - Optimize XGBoost parameters
3. **More Features** - Add weather, events, holidays
4. **Ensemble Methods** - Combine with Prophet/ARIMA for better accuracy
5. **Deployment** - Create API for police department integration